<a href="https://colab.research.google.com/github/X4Zero/SISTEMAS_DE_RECOMENDACION/blob/master/SistemaRecomendacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SISTEMAS DE RECOMENDACIÓN
Primera semana en Hackspace, el primer proyecto que desarrollaré será un sistema de recomendación.  
[Enlace del tutorial](https://www.datacamp.com/community/tutorials/recommender-systems-python)

De acuerdo al tutorial hay 3 tipos de sistemas de recomendación, los cuales son:


*   Recomendadores simples: básicamente hablando de películas recomiendan aquellas con mejores resultados en críticas, pues tienen mayor probabilidad de gustarle al público promedio
*   Recomendadores basados en contenido: usan metadatos de items anteriores que han gustado a los usuarios, teniendo la idea de que a los usuarios les gustarán items similares a los que le han gustado antes.
*   Motores de filtrado colaborativo: predicen la preferencia o  puntuación de un usuario sobre un item en base a preferencias o puntuaciones de otros usuarios.




## DATASET
Contiene metadata de 45000 películas. Full MovieLens Dataset.

El dataset contiene:


*   movies_metadata.csv: This file contains information on ~45,000 movies featured in the Full MovieLens dataset. Features include posters, backdrops, budget, genre, revenue, release dates, languages, production countries, and companies.
*   keywords.csv: Contains the movie plot keywords for our MovieLens movies. Available in the form of a stringified JSON Object.
*   credits.csv: Consists of Cast and Crew Information for all the movies. Available in the form of a stringified JSON Object.
*   links.csv: This file contains the TMDB and IMDB IDs of all the movies featured in the Full MovieLens dataset.
*   links_small.csv: Contains the TMDB and IMDB IDs of a small subset of 9,000 movies of the Full Dataset.
*   ratings_small.csv: The subset of 100,000 ratings from 700 users on 9,000 movies.


[Enlace del dataset completo](https://grouplens.org/datasets/movielens/latest/)

[Enlace del dataset usado para el trabajo](https://www.kaggle.com/rounakbanik/the-movies-dataset/data)

## RECOMENDADOR SIMPLE

In [12]:
# Import Pandas
import pandas as pd
import matplotlib.pyplot as plt

# Load Movies Metadata
metadata = pd.read_csv('/content/drive/My Drive/Colab Notebooks/HACKSPACE/SEMANA1/The_Movies_Dataset/movies_metadata.csv', low_memory=False)

# Print the first three rows
metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [6]:
registros, columnas = metadata.shape
print("{} registros".format(registros))
print("{} columnas".format(columnas))

45466 registros
24 columnas


Since you are trying to build a clone of IMDB's Top 250, let's use its weighted rating formula as a metric/score. Mathematically, it is represented as follows:

\begin{equation}
\text Weighted Rating (\bf WR) = \left({{\bf v} \over {\bf v} + {\bf m}} \cdot R\right) + \left({{\bf m} \over {\bf v} + {\bf m}} \cdot C\right)
\end{equation}

In the above equation,

v is the number of votes for the movie;

m is the minimum votes required to be listed in the chart;

R is the average rating of the movie;

C is the mean vote across the whole report.

You already have the values to v (vote_count) and R (vote_average) for each movie in the dataset. It is also possible to directly calculate C from this data.

In [7]:
# Calculate mean of vote average column
# La calificación promedio para una película en IMDB es de 5.61 en una escala de 0 a 10
C = metadata['vote_average'].mean()
print(C)

5.618207215133889


In [61]:
# Calculate the minimum number of votes required to be in the chart, m
# el número de votos recibidos por la película en el percentil 90
m = metadata['vote_count'].quantile(0.90)
print(m)

160.0


In [59]:
# estadísticas sobre la cantidad de votos
metadata['vote_count'].describe()

count    45460.000000
mean       109.897338
std        491.310374
min          0.000000
25%          3.000000
50%         10.000000
75%         34.000000
max      14075.000000
Name: vote_count, dtype: float64

In [64]:
# Filter out all qualified movies into a new DataFrame
q_movies = metadata.copy().loc[metadata['vote_count'] >= m]
q_movies.shape

(4555, 24)

In [65]:
metadata.shape

(45466, 24)

In [66]:
# Calcular el puntaje para cada película dentro de las seleccionadas
# Function that computes the weighted rating of each movie
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [67]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [68]:
#Sort movies based on score calculated above
q_movies = q_movies.sort_values('score', ascending=False)

#Print the top 15 movies
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(20)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.445869
834,The Godfather,6024.0,8.5,8.425439
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453
12481,The Dark Knight,12269.0,8.3,8.265477
2843,Fight Club,9678.0,8.3,8.256385
292,Pulp Fiction,8670.0,8.3,8.251406
522,Schindler's List,4436.0,8.3,8.206639
23673,Whiplash,4376.0,8.3,8.205404
5481,Spirited Away,3968.0,8.3,8.196055
2211,Life Is Beautiful,3643.0,8.3,8.187171


Podemos ver que hay varias coincidencias con los primeros puestos del top 250 de IMDB y el top 20 de películas resultado del recomendador simple